In [1]:
import cvxpy as cp
import scipy as sc
import numpy as np
import numpy.random as npr
import torch
from sklearn import datasets
from sklearn.cluster import KMeans
import pandas as pd
import lropt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import scipy.sparse as scs
import sys
# import ot
sys.path.append('..')
# from utils import plot_iters, plot_coverage_all
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "blue", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "red"]


## Setup Data

In [2]:
M = 5
l = 2
n = 8
m = 3
A = np.reshape(np.array([0.19,0.35,0.03,0.00,0.71,0.14,0.01,0.00,0.35,0.22,0.35,0.04,0.14,0.71,0.14,0.01,0.03,0.35,0.23,0.39,0.01,0.14,0.71,0.14,0.00,0.04,0.39,0.58,0.00,0.01,0.14,0.85,-1.28,0.44,0.12,0.01,0.19,0.35,0.03,0.00,0.44,-1.15,0.45,0.13,0.35,0.22,0.35,0.04,0.12,0.45,-1.15,0.57,0.03,0.35,0.23,0.39,0.01,0.13,0.57,-0.71,0.00,0.04,0.39,0.58]),(8,8))
B = np.reshape(np.array([0.39,0.00,-0.04,-0.39,0.04,-0.42,-0.04,0.39,-0.04,-0.00,-0.42,-0.00,0.57,0.01,-0.14,-0.58,0.13,-0.71,-0.13,0.57,-0.14,-0.01,-0.71,-0.01]),(8,3))
D = np.array([[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1]]).T

In [3]:
Q_ = {}
for i in range(M-1):
  Q_[i] = np.zeros((n,n))
Q_[4] = np.bmat([[np.eye(4),np.zeros((4,4))],[np.zeros((4,4)),np.zeros((4,4))]])

R_ = {}
for i in range(M):
  R_[i] = 10e-6*np.eye(m)

Q = sc.linalg.block_diag(*[Q_[i] for i in range(M)])
Q = scs.csr_matrix(Q)

R = sc.linalg.block_diag(*[R_[i] for i in range(M)])
R = scs.csr_matrix(R)

In [4]:
F = np.vstack([np.linalg.matrix_power(A,i) for i in range(1,M+1)])

In [5]:
G = {}
for i in range(M):
  firsthalf = np.hstack([np.linalg.matrix_power(A,j)@B for j in np.flip(range(i+1))])
  if M-i-1 > 0:
    secondhalf = np.hstack([np.zeros((n,m))]*(M-i-1))
  if i + 1 < M:
    G[i] = np.hstack([firsthalf,secondhalf])
  else: 
    G[i] = firsthalf
G = np.vstack([G[i] for i in range(M)])
G = scs.csr_matrix(G)

In [6]:
H = {}
for i in range(M):
  firsthalf = np.hstack([np.linalg.matrix_power(A,j)@D for j in np.flip(range(i+1))])
  if M-i-1 > 0:
    secondhalf = np.hstack([np.zeros((n,l))]*(M-i-1))
  if i + 1 < M:
    H[i] = np.hstack([firsthalf,secondhalf])
  else: 
    H[i] = firsthalf
H = np.vstack([H[i] for i in range(M)])

## Setup nominal problem

In [43]:
S = np.eye(m)
C = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[0,0,-1,1]])
C = np.hstack([C,np.zeros((4,4))])
y_bar = 1.8
u_bar = 1.8

In [44]:
N = 1500
np.random.seed(10)
data = np.random.normal(size=(N,l*M))
testdat = np.random.normal(size=(N,l*M))

In [45]:
K = 15
k_nums = [1,2,5,10,50,100,500,750,1000,1500]
kmeans = KMeans(n_clusters=K, n_init='auto').fit(data)
weights = np.bincount(kmeans.labels_) / N
gamma = cp.Variable(m*M)
theta = cp.Variable((M*m,M*l))
eta = cp.Variable()
lam = cp.Variable()
t = cp.Variable(K)

w = cp.Parameter(K)
dat = cp.Parameter((K,l*M))
epsilon = 0.2
rho = cp.Parameter()
rho.value = 0.01
w.value = weights
dat.value = kmeans.cluster_centers_
# outer = 0
# for k in data:
#   outer += np.outer(k,k)
# outer_exp = outer/(data.shape[0])
# obj = (F@np.zeros(n) + G@gamma).T@Q@(F@np.zeros(n) + G@gamma) + 2*(F@np.zeros(n) + G@gamma)@Q@((H+G@theta)@np.mean(data,axis=0)) + cp.trace(((H+G@theta).T@Q@(H+G@theta))@outer_exp) + gamma@R@gamma + 2*(gamma@R@theta)@np.mean(data,axis=0) + cp.trace((theta.T@R@theta)@outer_exp)


In [46]:
objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
constraints = [eta + lam*rho + w@t <= 0]
for i in range(M):
    constraints += [theta[i*m:(i+1)*m, i*l:]==0]
  
for i in range(M): 
    for j in range(m):
        constraints +=[cp.norm((1/epsilon)*S[j]@(theta[(i*m):(i+1)*m]))<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
for i in range(M): 
    for j in range(4):
        constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 
constraints += [t >=0]
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve(solver = "CLARABEL")

2.519197849129555e-09

In [48]:
evalval = 0
for k in range(testdat.shape[0]): 
    evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
evalval = eta.value + (evalval/testdat.shape[0])/epsilon

eval1 = 0
for k in range(testdat.shape[0]): 
    eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
eval1 = eval1/testdat.shape[0]

evalval, eval1

(1.994891101965555, -0.028125180426977307)

## Varying CVaR

In [59]:
S = np.eye(m)
C = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[0,0,-1,1]])
C = np.hstack([C,np.zeros((4,4))])
y_bar = 1.8
u_bar = 1.8

K = 15
k_nums = [1,2,5,10,50,100,500,750,1000,1500]
kmeans = KMeans(n_clusters=K, n_init='auto').fit(data)
weights = np.bincount(kmeans.labels_) / N
gamma = cp.Variable(m*M)
theta = cp.Variable((M*m,M*l))
eta = cp.Variable(M)
lam = cp.Variable()
t = cp.Variable(K)

w = cp.Parameter(K)
dat = cp.Parameter((K,l*M))
epsilon = np.array([0.1,0.2,0.2,0.2,0.2])
rho = cp.Parameter()
rho.value = 0.01
w.value = weights
dat.value = kmeans.cluster_centers_

In [60]:
objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
constraints = [lam*rho + w@t <= 0]
for i in range(M):
    constraints += [theta[i*m:(i+1)*m, i*l:]==0]
  
for i in range(M): 
    for j in range(m):
        constraints +=[cp.norm((1/epsilon[i])*S[j]@(theta[(i*m):(i+1)*m]))<= lam]
        for k in range(K):
            constraints += [eta[i] +(1/epsilon[i])*(-eta[i]  - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
            constraints += [eta[i] +(1/epsilon[i])*(-eta[i] - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

for i in range(M): 
    for j in range(4):
        constraints +=[cp.norm((1/epsilon[i])*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
        for k in range(K):
            constraints += [eta[i] + (1/epsilon[i])*(-eta[i]- y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
            constraints += [eta[i] + (1/epsilon[i])*(-eta[i]- y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 

for i in range(M):
    constraints += [t >= eta[i]]
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve(solver = "CLARABEL")

2.519197842676049e-09

In [61]:
evalval = 0
for k in range(testdat.shape[0]): 
    evalval += np.maximum(np.max([eta[i].value + np.maximum((np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar -eta[i].value),0)/epsilon[i] for i in range(M)]), np.max([eta[i].value + np.maximum((np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar - eta[i].value),0)/epsilon[i] for i in range(M)]))
evalval = evalval/testdat.shape[0]
eval1 = 0
for k in range(testdat.shape[0]): 
    eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
eval1 = eval1/testdat.shape[0]

evalval, eval1

(3.906252619839626, 0.024918549033982638)

## Varying f

In [49]:
S = {}
C = {}
inds = [1,0.9,0.8,0.7,0.6]
for i in range(M):
    S[i] = inds[i]*np.eye(m)
    temp = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[0,0,-1,1]])
    C[i] = inds[i]*np.hstack([temp,np.zeros((4,4))])
y_bar = 1.8
u_bar = 1.8

In [50]:
K = 15
k_nums = [1,2,5,10,50,100,500,750,1000,1500]
kmeans = KMeans(n_clusters=K, n_init='auto').fit(data)
weights = np.bincount(kmeans.labels_) / N
gamma = cp.Variable(m*M)
theta = cp.Variable((M*m,M*l))
eta = cp.Variable()
lam = cp.Variable()
t = cp.Variable(K)
w = cp.Parameter(K)
dat = cp.Parameter((K,l*M))
epsilon = 0.2
rho = cp.Parameter()
rho.value = 0.01
w.value = weights
dat.value = kmeans.cluster_centers_

In [51]:
objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
constraints = [eta + lam*rho + w@t <= 0]
for i in range(M):
    constraints += [theta[i*m:(i+1)*m, i*l:]==0]
  
for i in range(M): 
    for j in range(m):
        constraints +=[cp.norm((1/epsilon)*S[i][j]@(theta[(i*m):(i+1)*m]))<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - u_bar + S[i][j]@gamma[(i*m):(i+1)*m] + S[i][j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - u_bar - S[i][j]@gamma[(i*m):(i+1)*m] - S[i][j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

for i in range(M): 
    for j in range(4):
        constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - y_bar + C[i][j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[i][j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - y_bar - C[i][j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[i][j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 
constraints += [t>=0]
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve(solver = "CLARABEL")

2.519197872355496e-09

In [52]:
evalval = 0
for k in range(testdat.shape[0]): 
    evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C[i]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S[i]@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
evalval = eta.value + (evalval/testdat.shape[0])/epsilon

eval1 = 0
for k in range(testdat.shape[0]): 
    eval1 += np.maximum(np.max([np.linalg.norm(C[i]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S[i]@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
eval1 = eval1/testdat.shape[0]

evalval, eval1

(0.5570373871796714, -0.37903828750161034)

## Varying distance

In [67]:
scales = np.array([5,5,4,4,3,3,2,2,1,1])
scales = scales/np.mean(scales)
D_bar = np.diag(scales)
D_bar_inv = np.linalg.inv(D_bar)

In [68]:
S = np.eye(m)
C = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[0,0,-1,1]])
C = np.hstack([C,np.zeros((4,4))])
y_bar = 1.8
u_bar = 1.8

In [69]:
K = 15
k_nums = [1,2,5,10,50,100,500,750,1000,1500]
rescaled_dat = data@D_bar
kmeans = KMeans(n_clusters=K, n_init='auto').fit(rescaled_dat)
weights = np.bincount(kmeans.labels_) / N
gamma = cp.Variable(m*M)
theta = cp.Variable((M*m,M*l))
eta = cp.Variable()
lam = cp.Variable()
t = cp.Variable(K)

w = cp.Parameter(K)
dat = cp.Parameter((K,l*M))
epsilon = 0.2
rho = cp.Parameter()
rho.value = 0.001
w.value = weights
dat.value = kmeans.cluster_centers_@D_bar_inv

In [70]:
objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
constraints = [eta + lam*rho + w@t <= 0]
for i in range(M):
    constraints += [theta[i*m:(i+1)*m, i*l:]==0]
  
for i in range(M): 
    for j in range(m):
        constraints +=[cp.norm((1/epsilon)*S[j]@(theta[(i*m):(i+1)*m])@D_bar_inv)<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

for i in range(M): 
    for j in range(4):
        constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n])@D_bar_inv)<= lam]
        for k in range(K):
            constraints += [(1/epsilon)*(-eta - y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
            constraints += [(1/epsilon)*(-eta - y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 

constraints += [t >=0]
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve(solver = "CLARABEL")

2.5192025844038145e-09

In [71]:
evalval = 0
for k in range(testdat.shape[0]): 
    evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
evalval = eta.value + (evalval/testdat.shape[0])/epsilon

eval1 = 0
for k in range(testdat.shape[0]): 
    eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
eval1 = eval1/testdat.shape[0]

evalval, eval1

(1.698852247960835, -0.02646171617841436)

In [ ]:
def construct_nominal(epsilon,S,C,K, M=M,n=n,m=m,l=l):
    gamma = cp.Variable(m*M)
    theta = cp.Variable((M*m,M*l))
    eta = cp.Variable()
    lam = cp.Variable()
    t = cp.Variable(K)
    w = cp.Parameter(K)
    dat = cp.Parameter((K,l*M))
    rho = cp.Parameter()
    objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
    constraints = [eta + lam*rho + w@t <= 0]
    for i in range(M):
        constraints += [theta[i*m:(i+1)*m, i*l:]==0]
        
    for i in range(M): 
        for j in range(m):
            constraints +=[cp.norm((1/epsilon)*S[j]@(theta[(i*m):(i+1)*m]))<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
    for i in range(M): 
        for j in range(4):
            constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 
    constraints += [t >=0]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, gamma, theta, eta, lam, t, w, dat, rho

def eval_nominal(testdat,gamma, theta, eta, epsilon, S,C, M=M):
    evalval = 0
    for k in range(testdat.shape[0]): 
        evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
    evalval = eta.value + (evalval/testdat.shape[0])/epsilon

    eval1 = 0
    for k in range(testdat.shape[0]): 
        eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
    eval1 = eval1/testdat.shape[0]
    return evalval, eval1

In [ ]:
def construct_varycvar(epsilon,S,C,K, M=M,n=n,m=m,l=l):
    gamma = cp.Variable(m*M)
    theta = cp.Variable((M*m,M*l))
    eta = cp.Variable(M)
    lam = cp.Variable()
    t = cp.Variable(K)
    w = cp.Parameter(K)
    dat = cp.Parameter((K,l*M))
    rho = cp.Parameter()
    objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
    constraints = [lam*rho + w@t <= 0]
    for i in range(M):
        constraints += [theta[i*m:(i+1)*m, i*l:]==0]
      
    for i in range(M): 
        for j in range(m):
            constraints +=[cp.norm((1/epsilon[i])*S[j]@(theta[(i*m):(i+1)*m]))<= lam]
            for k in range(K):
                constraints += [eta[i] +(1/epsilon[i])*(-eta[i]  - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
                constraints += [eta[i] +(1/epsilon[i])*(-eta[i] - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

    for i in range(M): 
        for j in range(4):
            constraints +=[cp.norm((1/epsilon[i])*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
            for k in range(K):
                constraints += [eta[i] + (1/epsilon[i])*(-eta[i]- y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
                constraints += [eta[i] + (1/epsilon[i])*(-eta[i]- y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 

    for i in range(M):
        constraints += [t >= eta[i]]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, gamma, theta, eta, lam, t, w, dat, rho

def eval_cvar(testdat,gamma, theta, eta, epsilon, S,C, M=M):
    evalval = 0
    for k in range(testdat.shape[0]): 
        evalval += np.maximum(np.max([eta[i].value + np.maximum((np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar -eta[i].value),0)/epsilon[i] for i in range(M)]), np.max([eta[i].value + np.maximum((np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar - eta[i].value),0)/epsilon[i] for i in range(M)]))
    evalval = evalval/testdat.shape[0]
    eval1 = 0
    for k in range(testdat.shape[0]): 
        eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
    eval1 = eval1/testdat.shape[0]

    return evalval, eval1

In [ ]:
def construct_varyf(epsilon,S,C,K, M=M,n=n,m=m,l=l):
    gamma = cp.Variable(m*M)
    theta = cp.Variable((M*m,M*l))
    eta = cp.Variable()
    lam = cp.Variable()
    t = cp.Variable(K)
    w = cp.Parameter(K)
    dat = cp.Parameter((K,l*M))
    rho = cp.Parameter()
    objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
    constraints = [eta + lam*rho + w@t <= 0]
    for i in range(M):
        constraints += [theta[i*m:(i+1)*m, i*l:]==0]
      
    for i in range(M): 
        for j in range(m):
            constraints +=[cp.norm((1/epsilon)*S[i][j]@(theta[(i*m):(i+1)*m]))<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - u_bar + S[i][j]@gamma[(i*m):(i+1)*m] + S[i][j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - u_bar - S[i][j]@gamma[(i*m):(i+1)*m] - S[i][j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

    for i in range(M): 
        for j in range(4):
            constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n]))<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - y_bar + C[i][j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[i][j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - y_bar - C[i][j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[i][j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 
    constraints += [t>=0]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, gamma, theta, eta, lam, t, w, dat, rho

def eval_f(M,testdat,gamma, theta, eta, epsilon, S,C):
    evalval = 0
    for k in range(testdat.shape[0]): 
        evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C[i]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S[i]@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
    evalval = eta.value + (evalval/testdat.shape[0])/epsilon

    eval1 = 0
    for k in range(testdat.shape[0]): 
        eval1 += np.maximum(np.max([np.linalg.norm(C[i]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S[i]@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
    eval1 = eval1/testdat.shape[0]

    return evalval, eval1

In [ ]:
def construct_varydist(epsilon,S,C,K, D_bar_inv, M=M,n=n,m=m,l=l):
    gamma = cp.Variable(m*M)
    theta = cp.Variable((M*m,M*l))
    eta = cp.Variable()
    lam = cp.Variable()
    t = cp.Variable(K)
    w = cp.Parameter(K)
    dat = cp.Parameter((K,l*M))
    rho = cp.Parameter()
    objective = cp.quad_form(F@np.zeros(n) + G@(gamma + theta@np.mean(data,axis=0))+ H@np.mean(data,axis=0),Q) + cp.quad_form(gamma + theta@np.mean(data,axis=0),R)
    constraints = [eta + lam*rho + w@t <= 0]
    for i in range(M):
        constraints += [theta[i*m:(i+1)*m, i*l:]==0]
      
    for i in range(M): 
        for j in range(m):
            constraints +=[cp.norm((1/epsilon)*S[j]@(theta[(i*m):(i+1)*m])@D_bar_inv)<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - u_bar + S[j]@gamma[(i*m):(i+1)*m] + S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - u_bar - S[j]@gamma[(i*m):(i+1)*m] - S[j]@(theta[(i*m):(i+1)*m]@dat[k])) <= t[k]]

    for i in range(M): 
        for j in range(4):
            constraints +=[cp.norm((1/epsilon)*C[j]@((H+G@theta)[(i*n):(i+1)*n])@D_bar_inv)<= lam]
            for k in range(K):
                constraints += [(1/epsilon)*(-eta - y_bar + C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) + C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]]
                constraints += [(1/epsilon)*(-eta - y_bar - C[j]@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma)[(i*n):(i+1)*n]) - C[j]@(((H+G@theta)[(i*n):(i+1)*n])@dat[k])) <= t[k]] 

    constraints += [t >=0]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, gamma, theta, eta, lam, t, w, dat, rho

def eval_dist(testdat,gamma, theta, eta, epsilon, S,C, M=M):
    evalval = 0
    for k in range(testdat.shape[0]): 
        evalval += np.maximum(np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)])) - eta.value,0)
    evalval = eta.value + (evalval/testdat.shape[0])/epsilon

    eval1 = 0
    for k in range(testdat.shape[0]): 
        eval1 += np.maximum(np.max([np.linalg.norm(C@(F[(i*n):(i+1)*n]@np.zeros(n) + (G@gamma.value)[(i*n):(i+1)*n] + (H+G@theta.value)[(i*n):(i+1)*n]@testdat[k]),np.inf) - y_bar for i in range(M)]), np.max([np.linalg.norm(S@(gamma.value[(i*m):(i+1)*m] + theta.value[(i*m):(i+1)*m]@testdat[k]), np.inf) - u_bar for i in range(M)]))
    eval1 = eval1/testdat.shape[0]

    return evalval, eval1